# DPRP - Assignment 3, Natalia Nazarczuk 


# Reading data

In [1]:
import pandas as pd
import numpy as np

Badges = pd.read_csv("C:/Users/sebjan/OneDrive/Pulpit/Data Processing in R and Python/hw1/Badges.csv.gz",
compression = "gzip")
Comments = pd.read_csv("C:/Users/sebjan/OneDrive/Pulpit/Data Processing in R and Python/hw1/Comments.csv.gz",
compression = "gzip")
PostLinks = pd.read_csv("C:/Users/sebjan/OneDrive/Pulpit/Data Processing in R and Python/hw1/PostLinks.csv.gz",
compression = "gzip")
Posts = pd.read_csv("C:/Users/sebjan/OneDrive/Pulpit/Data Processing in R and Python/hw1/Posts.csv.gz",
compression = "gzip")
Tags = pd.read_csv("C:/Users/sebjan/OneDrive/Pulpit/Data Processing in R and Python/hw1/Tags.csv.gz",
compression = "gzip")
Users = pd.read_csv("C:/Users/sebjan/OneDrive/Pulpit/Data Processing in R and Python/hw1/Users.csv.gz",
compression = "gzip")
Votes = pd.read_csv("C:/Users/sebjan/OneDrive/Pulpit/Data Processing in R and Python/hw1/Votes.csv.gz",
compression = "gzip")

# Creating database

In [2]:
import os, os.path
import sqlite3
import tempfile

baza = os.path.join(tempfile.mkdtemp(), 'example.db')
if os.path.isfile(baza): 
    os.remove(baza)
    
conn = sqlite3.connect(baza)

Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

# Query 1

### SQL query

In [3]:
q1 = pd.read_sql_query("""SELECT
Name,
COUNT(*) AS Number,
MIN(Class) AS BestClass
FROM Badges
GROUP BY Name
ORDER BY Number DESC
LIMIT 10
""", conn)
q1

,Name,Number,BestClass
0,Autobiographer,24564,3
1,Student,23930,3
2,Supporter,17826,3
3,Popular Question,14840,3
4,Nice Answer,12406,3
5,Yearling,11281,2
6,Teacher,9746,3
7,Editor,9115,3
8,Notable Question,8613,2
9,Scholar,7224,3


### Pandas solution
We need add two essential columns 'Number' and 'BestClass' to our Badges dataframe using the function transform() while specifying the operations 'count' and 'min' accordingly. Then we extract the columns we want, drop duplicated rows and sort values descending by column 'Number'.

In [101]:
d1 = (Badges
      .assign(Number = Badges.groupby('Name')['Name'].transform('count'))
      .assign(BestClass =  Badges.groupby("Name")["Class"].transform("min")))

d1 = (d1[["Name", "Number", "BestClass"]]
      .drop_duplicates()
      .sort_values(by=["Number"], ascending=False)
      .reset_index(drop=True)
      .head(10))

d1

,Name,Number,BestClass
0,Autobiographer,24564,3
1,Student,23930,3
2,Supporter,17826,3
3,Popular Question,14840,3
4,Nice Answer,12406,3
5,Yearling,11281,2
6,Teacher,9746,3
7,Editor,9115,3
8,Notable Question,8613,2
9,Scholar,7224,3


In [100]:
q1.equals(d1)

True

## Query 2

### SQL query

In [6]:
q2 = pd.read_sql_query("""SELECT Location, COUNT(*) AS Count
FROM (
SELECT Posts.OwnerUserId, Users.Id, Users.Location
FROM Users
JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
""", conn)
q2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


### Pandas solution
After merging 'Users' and 'Posts' dataframes, we add a 'Count' column that holds the frequency of each 'Location' except for the empty ones. Then we extract the columns we want, drop duplicated rows and sort values descending by column 'Count'. At last, the type of 'Count' column has been changed from floats to integers.

In [96]:
d2 = pd.merge(Users, Posts, left_on='Id', right_on='OwnerUserId')
d2 = d2.assign(Count = d2.groupby('Location')['Location'].transform('count').loc[d2["Location"] != "NaN"])

d2 = (d2[["Location", "Count"]]
     .drop_duplicates()
     .dropna()
     .sort_values(by=["Count"], ascending=False)
     .reset_index(drop=True)
     .head(10))

d2["Count"] = d2["Count"].apply(int)

d2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


In [97]:
q2.equals(d2)

True

## Query 3 

### SQL query

In [45]:
q3 = pd.read_sql_query("""SELECT
    Users.AccountId,
    Users.DisplayName,
    Users.Location,
    AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM (
    SELECT
        AnsCount.AnswersCount,
        Posts.Id,
        Posts.OwnerUserId
    FROM (
        SELECT Posts.ParentId, COUNT(*) AS AnswersCount
        FROM Posts
        WHERE Posts.PostTypeId = 2
        GROUP BY Posts.ParentId
        ) AS AnsCount
    JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC
LIMIT 10
""", conn)
q3 = q3.sort_values(["AverageAnswersCount", "DisplayName"], ascending=[False, False]).reset_index(drop=True)
q3

,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,204.0,Josh,Australia,10.0
3,44093.0,Emma Arbogast,"Salem, OR",10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,79346.0,Thomas Matthews,California,8.0
6,42364.0,Petrogad,None,8.0
7,20473.0,Jeremy Boyd,"Houston, TX",8.0
8,19588.0,JD Isaacks,"Atlanta, GA",8.0
9,54571.0,Christian,None,8.0


### Pandas solution
Firstly, we selected only values for column 'ParentId' with 'PostTypeId' equal to 2 and added the column counting 'ParentId'. Dataframe 'PostAuth' holds merged 'Posts' with 'AnsCount' and the newly added column with average of answers count based on 'OwnerUserId'. Then we select the wanted columns and drop duplicated values. We have to sort values descending based on "AverageAnswersCount" and "DisplayName" in order for top 10 values to be in the same order as in sql solution. 

In [66]:
AnsCount = Posts.loc[Posts["PostTypeId"] == 2][["ParentId"]]
AnsCount = AnsCount.assign(AnswersCount = AnsCount.groupby('ParentId')['ParentId'].transform('count'))

PostAuth = pd.merge(Posts, AnsCount, left_on="Id", right_on="ParentId")
PostAuth = PostAuth[["AnswersCount", "Id", "OwnerUserId"]].drop_duplicates()
PostAuth = PostAuth.assign(AverageAnswersCount = PostAuth.groupby("OwnerUserId")["AnswersCount"].transform('mean'))

d3 = pd.merge(Users, PostAuth, left_on="AccountId", right_on="OwnerUserId")
d3 = (d3[["AccountId", "DisplayName", "Location", "AverageAnswersCount"]]
      .drop_duplicates()
      .sort_values(["AverageAnswersCount", "DisplayName"], ascending=[False, False])
      .reset_index(drop=True)
      .head(10))
d3

,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,204.0,Josh,Australia,10.0
3,44093.0,Emma Arbogast,"Salem, OR",10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,79346.0,Thomas Matthews,California,8.0
6,42364.0,Petrogad,NaN,8.0
7,20473.0,Jeremy Boyd,"Houston, TX",8.0
8,19588.0,JD Isaacks,"Atlanta, GA",8.0
9,54571.0,Christian,NaN,8.0


In [67]:
q3.equals(d3)

True

## Query 4

### SQL query

In [48]:
q4 = pd.read_sql_query("""
SELECT
    Posts.Title,
    UpVotesPerYear.Year,
    MAX(UpVotesPerYear.Count) AS Count
FROM (
        SELECT
            PostId,
            COUNT(*) AS Count,
            STRFTIME('%Y', Votes.CreationDate) AS Year
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, Year
    ) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year
ORDER BY Year ASC
""", conn)
q4

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,179
4,Immigration officer that stopped me at the air...,2015,117
5,I don't know my nationality. How can I visit D...,2016,134
6,Why prohibit engine braking?,2017,177
7,How can I find restaurants in the USA where ti...,2018,119
8,My name causes an issue with any booking! (nam...,2019,263
9,What's the longest distance that can be travel...,2020,110


### Pandas solution
First off we create a dataframe 'UpVotesPerYear' with votes of type 2 only, and then add two new columns 'Year' and 'Count' with extracted year from the date and the count of post ids per year respectively. The final dataframe containes 'Posts' and 'UpVotesPerYear' merged with posts of type 1. We sort them descending by 'Count' and take the highest one in each year.

In [133]:
UpVotesPerYear = Votes.loc[Votes["VoteTypeId"] == 2][["PostId", "CreationDate"]]
UpVotesPerYear = UpVotesPerYear.assign(Year = pd.to_datetime(UpVotesPerYear["CreationDate"]).dt.strftime("%Y"))
UpVotesPerYear = UpVotesPerYear.assign(Count = UpVotesPerYear.groupby(["PostId", "Year"]).transform("count"))

d4 = pd.merge(Posts, UpVotesPerYear, left_on="Id", right_on="PostId")
d4 = (d4[["Title", "Year", "Count"]]
      .loc[d4["PostTypeId"] == 1]
      .sort_values(by="Count", ascending=False)
      .groupby("Year")
      .head(1)
      .sort_values(by="Year", ascending=True)
      .reset_index(drop=True))

d4


,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,179
4,Immigration officer that stopped me at the air...,2015,117
5,I don't know my nationality. How can I visit D...,2016,134
6,Why prohibit engine braking?,2017,177
7,How can I find restaurants in the USA where ti...,2018,119
8,My name causes an issue with any booking! (nam...,2019,263
9,What's the longest distance that can be travel...,2020,110


In [134]:
q4.equals(d4)

True

## Query 5

### SQL query

In [3]:
q5 = pd.read_sql_query("""
SELECT
    Posts.Title,
    VotesByAge2.OldVotes
FROM Posts
JOIN (
    SELECT
        PostId,
        MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
        MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
        SUM(Total) AS Votes
    FROM (
        SELECT
            PostId,
            CASE STRFTIME('%Y', CreationDate)
                WHEN '2021' THEN 'new'
                WHEN '2020' THEN 'new'
                ELSE 'old'
                END VoteDate,
            COUNT(*) AS Total
        FROM Votes
        WHERE VoteTypeId IN (1, 2, 5)
        GROUP BY PostId, VoteDate
    ) AS VotesByAge
    GROUP BY VotesByAge.PostId
    HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10
""", conn)
q5.sort_values(by=["OldVotes","Title"], ascending=[False, False])
q5

,Title,OldVotes
0,How to avoid drinking vodka?,200
1,"How can I do a ""broad"" search for flights?",175
2,How to convince airport security that I am not...,119
3,Can I accidentally miss the in-flight food?,114
4,How can black travellers deal with unwanted ha...,108
5,Why are there no seat belts on trains?,106
6,"How do you avoid ""tourist traps"" when travelin...",104
7,Can I fly with a gold bar?,102
8,Best way to dispose of a carry-on bag in Hong ...,99
9,How to successfully haggle / bargain in markets,97


### Pandas solution
In the beginning, we create dataframe 'VotesByAge' which contains only votes of type 1, 2 or 5 and add new column 'Year' which holds extracted from 'CreationDate' year. Then, we create a column 'VoteDate' whch values are based on the year of the vote. The 'Total' holds the number of all votes together. The 'NewVotes' and 'OldVotes' columns shows the number of all votes of either 'new' and 'old' values. Then, grouping it by 'PostId' we can merge the rows and take out those that do not have any 'NewVotes'. In the final dataframe, we take only posts of type 1 and then we sort them descending by 'OldVotes', take out only important columns and drop duplicated values.

In [19]:
VotesByAge = Votes.loc[(Votes["VoteTypeId"] == 1) | (Votes["VoteTypeId"] == 2) | (Votes["VoteTypeId"] == 5)]
VotesByAge = VotesByAge.assign(Year = pd.to_datetime(VotesByAge["CreationDate"]).dt.strftime("%Y").astype(int))
VotesByAge = VotesByAge.assign(VoteDate = np.select([(VotesByAge["Year"] >= 2020) , (VotesByAge["Year"] < 2020)], ['new', 'old']))
VotesByAge = VotesByAge.groupby(["PostId", "VoteDate"]).size().reset_index(name='Total')

VotesByAge = VotesByAge.assign(NewVotes = VotesByAge.loc[VotesByAge["VoteDate"] == "new"]["Total"]).fillna(0)
VotesByAge = VotesByAge.assign(OldVotes = VotesByAge.loc[VotesByAge["VoteDate"] == "old"]["Total"]).fillna(0)
VotesByAge = VotesByAge[["PostId", "NewVotes", "OldVotes"]].groupby("PostId").sum().reset_index()
VotesByAge = VotesByAge.loc[(VotesByAge["NewVotes"] == 0)]

d5 = pd.merge(Posts, VotesByAge, left_on="Id", right_on="PostId")
d5 = d5.loc[d5["PostTypeId"] == 1]
d5["OldVotes"] = d5["OldVotes"].astype(np.int64) 
d5 = (d5[["Title", "OldVotes"]]
      .sort_values(by=["OldVotes","Title"], ascending=[False, False])
      .drop_duplicates()
      .reset_index(drop=True)
      .head(10))

d5

,Title,OldVotes
0,How to avoid drinking vodka?,200
1,"How can I do a ""broad"" search for flights?",175
2,How to convince airport security that I am not...,119
3,Can I accidentally miss the in-flight food?,114
4,How can black travellers deal with unwanted ha...,108
5,Why are there no seat belts on trains?,106
6,"How do you avoid ""tourist traps"" when travelin...",104
7,Can I fly with a gold bar?,102
8,Best way to dispose of a carry-on bag in Hong ...,99
9,How to successfully haggle / bargain in markets,97


In [20]:
d5.equals(q5)

True